# Strands Agents와 Amazon Bedrock Knowledge Bases를 활용한 Graph RAG
이 모듈에서는 **Strands Agents SDK**와 Amazon Bedrock Knowledge Bases를 결합하여 GraphRAG(Retrieval-Augmented Generation) 기반의 생성형 AI 애플리케이션을 구축하는 방법을 배웁니다. 이를 통해 그래프 데이터와 결합된 더욱 정확하고 맥락적이며 설명 가능한 응답을 생성할 수 있습니다.

다음 단계를 통해 Strands Agents와 Amazon Bedrock Knowledge Bases, GraphRAG를 사용하여 자율 AI 에이전트를 구축하는 과정을 안내합니다.

## 개요
**Strands Agents**는 AWS에서 출시한 오픈소스 SDK로, 모델 중심 접근 방식으로 자율 AI 에이전트를 구축하기 위해 설계되었습니다. Amazon Bedrock, AWS Lambda, AWS Step Functions 등 AWS 서비스와의 원활한 통합을 제공합니다.

Amazon Bedrock Knowledge Bases는 Amazon Neptune Analytics와 함께 완전 관리형 GraphRAG 기능을 제공합니다. 이 기능은 그래프와 결합된 RAG(Retrieval Augmented Generation) 기술을 사용하여 생성형 AI 애플리케이션을 향상시키므로 최종 사용자가 더욱 정확하고 포괄적인 응답을 얻을 수 있습니다.

GraphRAG는 Amazon Bedrock Knowledge Bases에 수집된 문서 전체에서 관련 엔티티 간의 관계와 구조적 요소(섹션 제목 등)를 자동으로 식별하고 활용합니다. 즉, 생성형 AI 애플리케이션이 여러 문서 청크에 걸쳐 데이터를 연결하고 추론해야 하는 경우 더욱 관련성 높은 응답을 제공할 수 있습니다.

### Strands Agents의 주요 기능:

- **모델 중심 설계**: 파운데이션 모델이 에이전트 인텔리전스의 핵심이라는 개념을 기반으로 구축되어 정교한 자율 추론 가능
- **MCP 통합**: Model Context Protocol(MCP)에 대한 네이티브 지원으로 LLM에 표준화된 컨텍스트 제공
- **AWS 서비스 통합**: Amazon Bedrock, AWS Lambda, AWS Step Functions 등과의 원활한 연결
- **멀티모달 지원**: 텍스트, 음성, 이미지 처리를 포함한 포괄적인 에이전트 상호작용
- **풍부한 도구 생태계**: AWS 서비스 상호작용을 위한 도구와 커스텀 도구 확장성

### GraphRAG를 활용한 Amazon Bedrock Knowledge Bases의 이점:

- 관련 엔티티 및 문서 섹션의 맥락 정보를 활용하여 **더욱 관련성 높은 응답** 제공
- 데이터 소스의 핵심 콘텐츠를 통합하고 불필요한 정보를 필터링하여 **향상된 요약** 제공
- 데이터셋의 서로 다른 엔티티 간 관계를 이해하고 출처를 제공하여 **더욱 설명 가능한 응답** 제공

**참고**: GraphRAG는 Amazon Bedrock Knowledge Bases와 Amazon Neptune Analytics가 모두 사용 가능한 AWS 리전에서 제공됩니다.

- 이 모듈에서는 Anthropic Claude 3 Haiku 모델을 파운데이션 모델로 사용합니다.

## 사전 요구사항
질문에 답변하기 전에 문서를 처리하여 Knowledge Base에 저장해야 합니다. 이 노트북에서는 `리테일 회사의 광고 캠페인 성과 보고서 데이터`를 사용하여 Amazon Bedrock Knowledge Bases를 생성합니다.

1. 문서(데이터 소스)를 Amazon S3 버킷에 업로드합니다.

## 설정

In [ ]:
# %pip install --upgrade pip --quiet
# %pip install -r requirements.txt --no-deps --quiet
# %pip install -r requirements.txt --upgrade --quiet

### boto3 클라이언트 및 Strands Agents 초기화
노트북 전체에서 Strands Agents의 retrieve 도구를 활용하여 Knowledge Base 기능을 테스트합니다.

In [ ]:
import json
import boto3
import pprint
import sys
from botocore.exceptions import ClientError
from botocore.client import Config
import time
import os
import logging

# Strands Agents imports
from strands import Agent
from strands_tools import retrieve
from strands.models import BedrockModel
    
from utils.knowledge_base import BedrockKnowledgeBase

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
account_id = sts_client.get_caller_identity()['Account']

# Create s3 client
s3_client = boto3.client('s3')

print(f"Region: {region_name}")
print(f"Account ID: {account_id}")

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-knowledge-base-{suffix}"
knowledge_base_description = "Graph RAG knowledge base with Strands Agents."

bucket_name = f'{knowledge_base_name}-{account_id}'

## Amazon Neptune을 벡터 스토어로 사용하는 KB 생성
GraphRAG를 구축하려면 벡터 스토어로 Amazon Neptune Analytics를 선택해야 합니다.
Knowledge Base는 문서 임베딩을 Amazon Neptune에 자동으로 생성하고 저장하며, 문서 말뭉치에서 파생된 엔티티와 그 관계의 그래프 표현도 함께 저장합니다.

GraphRAG 기반 애플리케이션이 실행되면 Strands Agents를 통해 Knowledge Base를 쿼리하여 최종 사용자에게 더욱 포괄적이고 관련성 높으며 설명 가능한 응답을 제공할 수 있습니다.

In [ ]:
data_sources = [{"type": "S3", "bucket_name": bucket_name}]

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "GRAPH", 
    suffix = f'{suffix}-f',
    vector_store="NEPTUNE_ANALYTICS" # can be OPENSEARCH_SERVERLESS or NEPTUNE_ANALYTICS
)

## Knowledge Base에 수집할 데이터 다운로드
다음 데이터를 사용합니다:

로컬 디렉토리에 저장된 합성 데이터를 첫 번째 데이터 소스로 사용
### S3 버킷 데이터 소스에 데이터 업로드

In [ ]:
import os
s3_client = boto3.client('s3')

def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("synthetic_dataset/", bucket_name)

## 수집(Ingestion) 작업 시작
KB와 데이터 소스가 생성되면 각 데이터 소스에 대한 수집 작업을 시작할 수 있습니다. 수집 작업 중에 KB는 데이터 소스의 문서를 가져와서 텍스트를 추출하도록 전처리하고, 제공된 청킹 크기를 기반으로 청킹하고, 각 청크의 임베딩을 생성한 다음 벡터 데이터베이스(이 경우 Neptune Analytics)에 기록합니다.

참고: 현재는 한 번에 하나의 수집 작업만 시작할 수 있습니다.

In [ ]:
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id
print(f"Knowledge Base ID: {kb_id}")

## Strands Agents 설정
이제 Strands Agents를 사용하여 Knowledge Base와 상호작용합니다. Strands Agents는 `retrieve` 도구를 통해 Amazon Bedrock Knowledge Base에서 정보를 검색할 수 있습니다.

## 옵션: 기존 Knowledge Base 사용하기

**이미 생성된 Knowledge Base가 있나요?** 그렇다면 위의 KB 생성 과정을 건너뛰고 여기서부터 시작할 수 있습니다!

기존 KB ID만 있으면 바로 Strands Agent를 사용할 수 있습니다. 아래 셀의 주석을 해제하고 KB ID를 입력하세요.

In [ ]:
# 기존 Knowledge Base를 사용하려면 아래 주석을 해제하고 KB ID를 입력하세요
# kb_id = "YOUR_EXISTING_KB_ID_HERE"  # 예: "ABCDEFGHIJ"
# print(f"기존 Knowledge Base ID 사용: {kb_id}")

# KB가 위에서 새로 생성되었다면 이 셀은 건너뛰세요

### 기존 KB를 사용하는 경우의 장점

기존 Knowledge Base가 이미 있다면:
- ✅ **즉시 시작**: KB 생성 및 데이터 수집 시간(수십 분) 절약
- ✅ **기존 데이터 활용**: 이미 수집된 데이터를 바로 쿼리 가능
- ✅ **비용 절감**: 새로운 리소스 생성 없이 기존 인프라 활용
- ✅ **프로덕션 데이터**: 실제 프로덕션 환경의 KB로 테스트 가능

**참고**: 기존 KB ID는 AWS 콘솔의 Amazon Bedrock > Knowledge bases 섹션에서 확인할 수 있습니다.

In [ ]:
# Set environment variables for Strands Agents
os.environ['KNOWLEDGE_BASE_ID'] = kb_id
os.environ['AWS_REGION'] = region_name
os.environ['AWS_DEFAULT_REGION'] = region_name

print(f"Strands Agent configured with Knowledge Base ID: {kb_id}")

## Strands Agents로 Knowledge Graph 테스트
이제 Strands Agents를 사용하여 Amazon Bedrock Knowledge Bases로 생성된 `Neptune Analytics` 그래프를 쿼리할 수 있습니다.

### Strands Agent로 Knowledge Base 질의
Strands Agent는 `retrieve` 도구를 사용하여 Knowledge Base에서 정보를 검색하고, 파운데이션 모델을 사용하여 자연스러운 응답을 생성합니다.

`query = 2023년 4분기 온라인 광고 캠페인의 전체 성과를 요약해줘.`

이 쿼리에 대한 정답은 다음과 유사해야 합니다:

```
2023년 4분기 온라인 광고 캠페인 성과 요약:
- 총 광고 지출: 15억 원
- 총 노출수: 5,000만 회
- 총 클릭수: 250만 회
- 평균 CTR(클릭률): 5.0%
- 총 전환수: 12만 5천 건
- 전환율: 5.0%
- ROAS(광고 투자 대비 수익): 350%
```

**참고**: 응답 생성에 선택한 모델에 따라 실제 응답은 다를 수 있지만 의미상으로는 유사합니다.

In [ ]:
# Strands용 베드락 모델 생성
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.1,
    streaming=False
)
# retrieve tool과 함께 Strands Agent 생성
agent = Agent(
    model=model,
    tools=[retrieve],
    system_prompt="""당신은 광고 캠페인 성과 분석 전문가입니다. 
    Knowledge Base에서 정보를 검색하여 사용자의 질문에 정확하고 포괄적으로 답변하세요.
    검색된 정보를 기반으로 명확하고 구조화된 답변을 제공하고, 가능한 경우 숫자와 통계를 포함하세요.
    답변은 한국어로 제공하세요."""
)

print("Strands Agent가 성공적으로 초기화되었습니다.")
print(f"사용 모델: {model_id}")
print(f"사용 도구: retrieve (Knowledge Base ID: {kb_id})")

In [ ]:
query = "2023년 4분기 온라인 광고 캠페인의 전체 성과를 요약해줘."

In [ ]:
# Query using Strands Agent
print(f"질문: {query}\n")
print("="*80)

response = agent(query)

print("\n답변:")
print("="*80)
print(response)

보시다시피 Strands Agent를 사용하면 자연스러운 대화형 응답을 얻을 수 있습니다. Agent는 내부적으로 `retrieve` 도구를 사용하여 Knowledge Base를 쿼리하고, 검색된 정보를 바탕으로 응답을 생성합니다.

### Strands Agent로 추가 질의하기
이제 더 구체적인 질문을 해보겠습니다.

In [ ]:
query2 = "모바일 앱 광고 캠페인에서 가장 높은 전환율을 기록한 광고 소재는 무엇인가요?"

print(f"질문: {query2}\n")
print("="*80)

response2 = agent(query2)

print("\n답변:")
print("="*80)
print(response2)

### Strands Agent의 멀티턴 대화
Strands Agent는 이전 대화 맥락을 유지할 수 있습니다. 이전 질문과 관련된 후속 질문을 해보겠습니다.

In [ ]:
query3 = "그렇다면 다른 광고 채널들과 비교했을 때 모바일 앱 광고의 성과는 어떤가요?"

print(f"질문: {query3}\n")
print("="*80)

response3 = agent(query3)

print("\n답변:")
print("="*80)
print(response3)

### 복잡한 분석 질의
GraphRAG의 장점을 활용하여 여러 엔티티 간의 관계를 분석하는 복잡한 질문을 해보겠습니다.

In [ ]:
query4 = "광고 지출 대비 가장 높은 ROAS를 기록한 캠페인은 무엇이며, 그 이유는 무엇인가요?"

print(f"질문: {query4}\n")
print("="*80)

response4 = agent(query4)

print("\n답변:")
print("="*80)
print(response4)

### 추천 및 인사이트 질의
마지막으로 Agent가 데이터를 분석하여 비즈니스 인사이트와 추천을 제공하도록 해보겠습니다.

In [ ]:
query5 = "2024년 1분기 광고 예산을 어떻게 배분하면 좋을지 2023년 4분기 데이터를 기반으로 추천해주세요."

print(f"질문: {query5}\n")
print("="*80)

response5 = agent(query5)

print("\n답변:")
print("="*80)
print(response5)

## Strands Agents의 장점

이 노트북에서 Strands Agents를 사용하여 GraphRAG 기반 Knowledge Base와 상호작용하는 방법을 살펴보았습니다. Strands Agents는 다음과 같은 장점을 제공합니다:

1. **간편한 통합**: 몇 줄의 코드로 Amazon Bedrock Knowledge Base와 통합 가능
2. **자율적 추론**: Agent가 자동으로 적절한 도구(retrieve)를 선택하고 사용
3. **자연스러운 대화**: 멀티턴 대화를 지원하여 이전 맥락을 유지
4. **확장성**: 필요에 따라 추가 도구와 기능을 쉽게 통합 가능
5. **AWS 네이티브**: AWS 서비스와의 원활한 통합으로 엔터프라이즈급 보안 및 확장성 제공

## 리소스 정리
모든 리소스를 삭제하려면 아래 섹션의 주석을 제거하고 실행하세요.

In [ ]:
# 역할 및 정책 삭제
# print("===============================Knowledge Base 및 관련 리소스 삭제 중==============================\n")
# knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

<div class="alert alert-block alert-warning">
<b>참고:</b> 요금 발생을 방지하려면 KB, Neptune Analytics 그래프 및 관련 IAM 역할과 정책을 삭제하는 것을 잊지 마세요.
</div>

## 추가 리소스

- [Strands Agents 공식 문서](https://strandsagents.com/latest/)
- [Strands Agents GitHub](https://github.com/strands-agents/)
- [AWS 블로그: Introducing Strands Agents](https://aws.amazon.com/blogs/opensource/introducing-strands-agents-an-open-source-ai-agents-sdk/)
- [Amazon Bedrock Knowledge Bases 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)
- [GraphRAG with Amazon Bedrock](https://aws.amazon.com/blogs/machine-learning/build-graphrag-applications-using-amazon-bedrock-knowledge-bases/)